# 4. Edited Nearest Neighbours
소수 클래스 주변의 다중 클래스 값을 제거하는 방법론

In [3]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

NameError: name 'type_check' is not defined

In [ ]:
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, EditedNearestNeighbours
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("./데이터자료/9. 데이터셋.csv")
feature = pd.read_csv("./데이터자료/8. feature.csv")

In [ ]:
year = df["회계년"].unique().tolist()
year

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [ ]:
col = df.columns[7:55]
x_train = []
y_train = []
x_test = []
y_test = []

In [ ]:
# train = df[df["회계년"] == 2017]
# x_train_year = train[col]
# y_train_year = train["label"]
# x_train_year, y_train_year = RandomUnderSampler(random_state=4,sampling_strategy=0.1).fit_resample(x_train_year,y_train_year)
# x_train.append(x_train_year)
# y_train.append(y_train_year)

In [ ]:
col = df.columns[7:55]
x_train = []
y_train = []
x_test = []
y_test = []
for i in year:
    if i <= 2018:
        print(i)
        train = df[df["회계년"] == i].reset_index()
        x_train_year = train[col]
        y_train_year = train["label"]
        x_train_year, y_train_year = EditedNearestNeighbours(kind_sel="all", n_neighbors=3).fit_resample(x_train_year,y_train_year)
        x_train.append(x_train_year)
        y_train.append(y_train_year)
 
    else:
        test = df[df["회계년"] == i].reset_index()
        x_test_year = test[col]
        y_test_year = test["label"]
        x_test_year, y_test_year = EditedNearestNeighbours(kind_sel="all", n_neighbors=3).fit_resample(x_test_year,y_test_year)
        x_test.append(x_test_year)
        y_test.append(y_test_year)

2011
2012
2013
2014
2015
2016
2017
2018


In [ ]:
x_train = pd.concat(x_train).reset_index(drop=True)
y_train = pd.concat(y_train).reset_index(drop=True)
x_test = pd.concat(x_test).reset_index(drop=True)
y_test = pd.concat(y_test).reset_index(drop=True)

In [ ]:
y_train

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6760    1.0
6761    1.0
6762    1.0
6763    1.0
6764    1.0
Name: label, Length: 6765, dtype: float64

In [ ]:
mda_feature=feature["mda_feature"].tolist()
logit_feature=feature["logit_feature"].tolist()
lasso_feature=feature["lasso_feature"].tolist()
ela_feature=feature["ela_feature"].tolist()
same_feature=feature["same_feature"].tolist()

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [ ]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")


In [ ]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [ ]:
mda_x_train = x_train[mda_feature].reset_index(drop='index')
mda_x_test = x_test[mda_feature].reset_index(drop='index')
mda_y_train = y_train
mda_y_test = y_test 

In [ ]:
logit_x_train = x_train[logit_feature].reset_index(drop='index')
logit_x_test = x_test[logit_feature].reset_index(drop='index')
logit_y_train = y_train
logit_y_test = y_test 

In [ ]:
lasso_x_train = x_train[lasso_feature].reset_index(drop='index')
lasso_x_test = x_test[lasso_feature].reset_index(drop='index')
lasso_y_train = y_train
lasso_y_test = y_test 

In [ ]:
ela_x_train = x_train[ela_feature].reset_index(drop='index')
ela_x_test = x_test[ela_feature].reset_index(drop='index')
ela_y_train = y_train
ela_y_test = y_test 

In [ ]:
same_x_train = x_train[same_feature].reset_index(drop='index')
same_x_test = x_test[same_feature].reset_index(drop='index')
same_y_train = y_train
same_y_test = y_test 

In [ ]:
fea_x_train = [mda_x_train, logit_x_train, lasso_x_train, ela_x_train, same_x_train]
fea_y_train = [mda_y_train, logit_y_train, lasso_y_train, ela_y_train, same_y_train]
fea_x_test = [mda_x_test, logit_x_test, lasso_x_test, ela_x_test, same_x_test]
fea_y_test = [mda_y_test, logit_y_test, lasso_y_test, ela_y_test, same_y_test]
fea = ["mda","logit","lasso","ela","same"]

In [ ]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lda_clf.fit(x_train, y_train)
    lda_pred = lda_clf.predict(x_test)
    lda_pred_proba = lda_clf.predict_proba(x_test)
    print(feature,"lda") 
    get_clf_eval(y_test, lda_pred)
    get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)



mda lda
오차행렬
[[2176   53]
 [  36   14]]
정확도: 0.9609, 정밀도 : 0.2090, 재현율:0.2800,F1 스코어:0.2393
ROC 스코어: 0.6281, PR score : 0.0743
임곗값: 0.1
오차행렬
[[2115  114]
 [  28   22]]
정확도: 0.9377, 정밀도 : 0.1618, 재현율:0.4400,F1 스코어:0.2366
ROC 스코어: 0.6944, PR score : 0.0835
임곗값: 0.2
오차행렬
[[2141   88]
 [  29   21]]
정확도: 0.9487, 정밀도 : 0.1927, 재현율:0.4200,F1 스코어:0.2642
ROC 스코어: 0.6903, PR score : 0.0936
임곗값: 0.3
오차행렬
[[2158   71]
 [  31   19]]
정확도: 0.9552, 정밀도 : 0.2111, 재현율:0.3800,F1 스코어:0.2714
ROC 스코어: 0.6741, PR score : 0.0938
임곗값: 0.4
오차행렬
[[2169   60]
 [  36   14]]
정확도: 0.9579, 정밀도 : 0.1892, 재현율:0.2800,F1 스코어:0.2258
ROC 스코어: 0.6265, PR score : 0.0688
임곗값: 0.5
오차행렬
[[2176   53]
 [  36   14]]
정확도: 0.9609, 정밀도 : 0.2090, 재현율:0.2800,F1 스코어:0.2393
ROC 스코어: 0.6281, PR score : 0.0743
임곗값: 0.6
오차행렬
[[2182   47]
 [  37   13]]
정확도: 0.9631, 정밀도 : 0.2167, 재현율:0.2600,F1 스코어:0.2364
ROC 스코어: 0.6195, PR score : 0.0726
임곗값: 0.7
오차행렬
[[2184   45]
 [  38   12]]
정확도: 0.9636, 정밀도 : 0.2105, 재현율:0.2400,F1 스코어:0.2243
ROC 스코어: 0.6

In [ ]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    dt_clf.fit(x_train, y_train)
    dt_pred = dt_clf.predict(x_test)
    dt_pred_proba = dt_clf.predict_proba(x_test)   
    print(feature,"디시전트리") 
    get_clf_eval(y_test, dt_pred)
    get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 디시전트리
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.1
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.2
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.3
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.4
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.5
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.6
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0.6113, PR score : 0.0731
임곗값: 0.7
오차행렬
[[2190   39]
 [  38   12]]
정확도: 0.9662, 정밀도 : 0.2353, 재현율:0.2400,F1 스코어:0.2376
ROC 스코어: 0

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    # key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
    param_grid = {
      'criterion': ['entropy', 'gini'],
               'max_depth': [2,4,6,8],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4,6,8,10],
               'min_samples_split': [4,6,8,10]}
    grid_search = GridSearchCV(tree
                            , param_grid = param_grid
                            , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                            , n_jobs = -1)  #-1: 모든 코어 사용

    grid_search.fit(x_train, y_train)

    # 최선의 결과 출력
    print('피처:{2} 최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_, feature))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
    dt_clf_best.fit(x_train, y_train)
    dt_best_pred = dt_clf_best.predict(x_test)
    dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, dt_best_pred)
    get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


피처:mda 최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 8, 'min_samples_split': 4}, 최적 평균 정확도 :0.984
mda 디시전트리 파라미터적용
오차행렬
[[2226    3]
 [  42    8]]
정확도: 0.9803, 정밀도 : 0.7273, 재현율:0.1600,F1 스코어:0.2623
ROC 스코어: 0.5793, PR score : 0.1348
임곗값: 0.1
오차행렬
[[2184   45]
 [  32   18]]
정확도: 0.9662, 정밀도 : 0.2857, 재현율:0.3600,F1 스코어:0.3186
ROC 스코어: 0.6699, PR score : 0.1169
임곗값: 0.2
오차행렬
[[2184   45]
 [  32   18]]
정확도: 0.9662, 정밀도 : 0.2857, 재현율:0.3600,F1 스코어:0.3186
ROC 스코어: 0.6699, PR score : 0.1169
임곗값: 0.3
오차행렬
[[2216   13]
 [  37   13]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.2600,F1 스코어:0.3421
ROC 스코어: 0.6271, PR score : 0.1462
임곗값: 0.4
오차행렬
[[2220    9]
 [  39   11]]
정확도: 0.9789, 정밀도 : 0.5500, 재현율:0.2200,F1 스코어:0.3143
ROC 스코어: 0.6080, PR score : 0.1381
임곗값: 0.5
오차행렬
[[2226    3]
 [  42    8]]
정확도: 0.9803, 정밀도 : 0.7273, 재현율:0.1600,F1 스코어:0.2623
ROC 스코어: 0.5793, PR score : 0.1348
임곗값: 0.6
오차행렬
[[2226    3]
 [  42    8]]
정확도: 0.9803, 정밀도 : 0.7273, 재현율:0.

In [ ]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    rf_clf.fit(x_train, y_train)
    rf_pred = rf_clf.predict(x_test)
    rf_pred_proba = rf_clf.predict_proba(x_test)
    print(feature,"랜덤포레스트") 
    get_clf_eval(y_test, rf_pred)
    get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 랜덤포레스트
오차행렬
[[2221    8]
 [  42    8]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.1600,F1 스코어:0.2424
ROC 스코어: 0.5782, PR score : 0.0984
임곗값: 0.1
오차행렬
[[2131   98]
 [  24   26]]
정확도: 0.9465, 정밀도 : 0.2097, 재현율:0.5200,F1 스코어:0.2989
ROC 스코어: 0.7380, PR score : 0.1196
임곗값: 0.2
오차행렬
[[2193   36]
 [  29   21]]
정확도: 0.9715, 정밀도 : 0.3684, 재현율:0.4200,F1 스코어:0.3925
ROC 스코어: 0.7019, PR score : 0.1675
임곗값: 0.3
오차행렬
[[2210   19]
 [  34   16]]
정확도: 0.9767, 정밀도 : 0.4571, 재현율:0.3200,F1 스코어:0.3765
ROC 스코어: 0.6557, PR score : 0.1612
임곗값: 0.4
오차행렬
[[2217   12]
 [  39   11]]
정확도: 0.9776, 정밀도 : 0.4783, 재현율:0.2200,F1 스코어:0.3014
ROC 스코어: 0.6073, PR score : 0.1223
임곗값: 0.5
오차행렬
[[2221    8]
 [  42    8]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.1600,F1 스코어:0.2424
ROC 스코어: 0.5782, PR score : 0.0984
임곗값: 0.6
오차행렬
[[2225    4]
 [  43    7]]
정확도: 0.9794, 정밀도 : 0.6364, 재현율:0.1400,F1 스코어:0.2295
ROC 스코어: 0.5691, PR score : 0.1080
임곗값: 0.7
오차행렬
[[2226    3]
 [  45    5]]
정확도: 0.9789, 정밀도 : 0.6250, 재현율:0.1000,F1 스코어:0.1724
ROC 스코어: 

In [ ]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    grid_search = {'criterion': ['entropy', 'gini'],
                'max_depth': [2,4,6,8],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [4,6,8,10],
                'min_samples_split': [4,6,8,10],
                'n_estimators': [20,40,60,80,100]}

    model = RandomForestClassifier(random_state=52)
    grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                                cv = 5,verbose=0, n_jobs = -1)
    grid_search.fit(x_train,y_train)

    # 최선의 결과 출력
    print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
    rf_clf_best.fit(x_train, y_train)
    rf_best_pred = rf_clf_best.predict(x_test)
    rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, rf_best_pred)
    get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 6, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 60}, 최적 평균 정확도 :0.985
mda 디시전트리 파라미터적용
오차행렬
[[2226    3]
 [  41    9]]
정확도: 0.9807, 정밀도 : 0.7500, 재현율:0.1800,F1 스코어:0.2903
ROC 스코어: 0.5893, PR score : 0.1530
임곗값: 0.1
오차행렬
[[2167   62]
 [  27   23]]
정확도: 0.9609, 정밀도 : 0.2706, 재현율:0.4600,F1 스코어:0.3407
ROC 스코어: 0.7161, PR score : 0.1363
임곗값: 0.2
오차행렬
[[2204   25]
 [  35   15]]
정확도: 0.9737, 정밀도 : 0.3750, 재현율:0.3000,F1 스코어:0.3333
ROC 스코어: 0.6444, PR score : 0.1279
임곗값: 0.3
오차행렬
[[2216   13]
 [  37   13]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.2600,F1 스코어:0.3421
ROC 스코어: 0.6271, PR score : 0.1462
임곗값: 0.4
오차행렬
[[2224    5]
 [  40   10]]
정확도: 0.9803, 정밀도 : 0.6667, 재현율:0.2000,F1 스코어:0.3077
ROC 스코어: 0.5989, PR score : 0.1509
임곗값: 0.5
오차행렬
[[2226    3]
 [  41    9]]
정확도: 0.9807, 정밀도 : 0.7500, 재현율:0.1800,F1 스코어:0.2903
ROC 스코어: 0.5893, PR score : 0.1530
임곗값: 0.6
오차행렬
[[2226    3]
 [  43    7]]
정확도: 0.9798, 정밀도 : 0

In [ ]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    lr_pred_proba = lr_clf.predict_proba(x_test)
    print(feature,"로지스틱회귀") 
    get_clf_eval(y_test, lr_pred)
    get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

mda 로지스틱회귀
오차행렬
[[2227    2]
 [  48    2]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.0400,F1 스코어:0.0741
ROC 스코어: 0.5196, PR score : 0.0411
임곗값: 0.1
오차행렬
[[2182   47]
 [  28   22]]
정확도: 0.9671, 정밀도 : 0.3188, 재현율:0.4400,F1 스코어:0.3697
ROC 스코어: 0.7095, PR score : 0.1526
임곗값: 0.2
오차행렬
[[2212   17]
 [  39   11]]
정확도: 0.9754, 정밀도 : 0.3929, 재현율:0.2200,F1 스코어:0.2821
ROC 스코어: 0.6062, PR score : 0.1035
임곗값: 0.3
오차행렬
[[2224    5]
 [  43    7]]
정확도: 0.9789, 정밀도 : 0.5833, 재현율:0.1400,F1 스코어:0.2258
ROC 스코어: 0.5689, PR score : 0.1005
임곗값: 0.4
오차행렬
[[2226    3]
 [  47    3]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.0600,F1 스코어:0.1071
ROC 스코어: 0.5293, PR score : 0.0506
임곗값: 0.5
오차행렬
[[2227    2]
 [  48    2]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.0400,F1 스코어:0.0741
ROC 스코어: 0.5196, PR score : 0.0411
임곗값: 0.6
오차행렬
[[2228    1]
 [  50    0]]
정확도: 0.9776, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4998, PR score : 0.0219
임곗값: 0.7
오차행렬
[[2228    1]
 [  50    0]]
정확도: 0.9776, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 

In [ ]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {'penalty': ['l2','l1','elasticnet','None'],
            'C':[0.01,0.1,1,3,5,10]}

    grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
    grid_search.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
    lr_clf_best.fit(x_train, y_train)
    lr_best_pred = lr_clf_best.predict(x_test)
    lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

    print(feature,"로지스틱회귀 파라미터적용") 
    get_clf_eval(y_test, lr_best_pred)
    get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 3, 'penalty': 'l2'}, 최적 평균 정확도:0.984
mda 로지스틱회귀 파라미터적용
오차행렬
[[2226    3]
 [  46    4]]
정확도: 0.9785, 정밀도 : 0.5714, 재현율:0.0800,F1 스코어:0.1404
ROC 스코어: 0.5393, PR score : 0.0659
임곗값: 0.1
오차행렬
[[2176   53]
 [  28   22]]
정확도: 0.9645, 정밀도 : 0.2933, 재현율:0.4400,F1 스코어:0.3520
ROC 스코어: 0.7081, PR score : 0.1414
임곗값: 0.2
오차행렬
[[2207   22]
 [  33   17]]
정확도: 0.9759, 정밀도 : 0.4359, 재현율:0.3400,F1 스코어:0.3820
ROC 스코어: 0.6651, PR score : 0.1627
임곗값: 0.3
오차행렬
[[2219   10]
 [  40   10]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.2000,F1 스코어:0.2857
ROC 스코어: 0.5978, PR score : 0.1176
임곗값: 0.4
오차행렬
[[2224    5]
 [  43    7]]
정확도: 0.9789, 정밀도 : 0.5833, 재현율:0.1400,F1 스코어:0.2258
ROC 스코어: 0.5689, PR score : 0.1005
임곗값: 0.5
오차행렬
[[2226    3]
 [  46    4]]
정확도: 0.9785, 정밀도 : 0.5714, 재현율:0.0800,F1 스코어:0.1404
ROC 스코어: 0.5393, PR score : 0.0659
임곗값: 0.6
오차행렬
[[2227    2]
 [  49    1]]
정확도: 0.9776, 정밀도 : 0.3333, 재현율:0.0200,F1 스코어:0.0377
ROC 스코어: 0.5096, PR score : 0.0282
임곗값: 0.7
오차행렬
[[2227    2]
 [  50    0]]

In [ ]:
# 1-5 KNN 
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    knn_clf.fit(x_train, y_train)
    knn_pred = knn_clf.predict(x_test)
    knn_pred_proba = knn_clf.predict_proba(x_test)
    print(feature,"KNN") 
    get_clf_eval(y_test, knn_pred)
    get_eval_by_threshold(y_test , knn_pred_proba[:,1].reshape(-1,1), thresholds)



mda KNN
오차행렬
[[2224    5]
 [  41    9]]
정확도: 0.9798, 정밀도 : 0.6429, 재현율:0.1800,F1 스코어:0.2812
ROC 스코어: 0.5889, PR score : 0.1337
임곗값: 0.1
오차행렬
[[2118  111]
 [  26   24]]
정확도: 0.9399, 정밀도 : 0.1778, 재현율:0.4800,F1 스코어:0.2595
ROC 스코어: 0.7151, PR score : 0.0967
임곗값: 0.2
오차행렬
[[2216   13]
 [  37   13]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.2600,F1 스코어:0.3421
ROC 스코어: 0.6271, PR score : 0.1462
임곗값: 0.3
오차행렬
[[2216   13]
 [  37   13]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.2600,F1 스코어:0.3421
ROC 스코어: 0.6271, PR score : 0.1462
임곗값: 0.4
오차행렬
[[2224    5]
 [  41    9]]
정확도: 0.9798, 정밀도 : 0.6429, 재현율:0.1800,F1 스코어:0.2812
ROC 스코어: 0.5889, PR score : 0.1337
임곗값: 0.5
오차행렬
[[2224    5]
 [  41    9]]
정확도: 0.9798, 정밀도 : 0.6429, 재현율:0.1800,F1 스코어:0.2812
ROC 스코어: 0.5889, PR score : 0.1337
임곗값: 0.6
오차행렬
[[2227    2]
 [  45    5]]
정확도: 0.9794, 정밀도 : 0.7143, 재현율:0.1000,F1 스코어:0.1754
ROC 스코어: 0.5496, PR score : 0.0912
임곗값: 0.7
오차행렬
[[2227    2]
 [  45    5]]
정확도: 0.9794, 정밀도 : 0.7143, 재현율:0.1000,F1 스코어:0.1754
ROC 스코어: 0.5

In [ ]:
# knn 파라미터 구하기
from sklearn.metrics import accuracy_score
acc = []
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    for k in range(1,20):
        knn_clf = KNeighborsClassifier(n_neighbors=k)
        knn_clf.fit(x_train, y_train)
        knn_pred = knn_clf.predict(x_test)
        knn_score=accuracy_score(y_test,knn_pred)

        acc.append(knn_score)
    int = acc.index(max(acc))+1
    print("최적거리")
    print(int)

    knn_clf_best =KNeighborsClassifier(n_neighbors=int)
    knn_clf_best.fit(x_train, y_train)
    knn_best_pred = knn_clf_best.predict(x_test)
    knn_best_pred_proba = knn_clf_best.predict_proba(x_test)   

    print(feature,"KNN 파라미터적용") 
    get_clf_eval(y_test, knn_best_pred)
    get_eval_by_threshold(y_test ,knn_best_pred_proba[:,1].reshape(-1,1), thresholds)      


최적거리
11
mda KNN 파라미터적용
오차행렬
[[2226    3]
 [  41    9]]
정확도: 0.9807, 정밀도 : 0.7500, 재현율:0.1800,F1 스코어:0.2903
ROC 스코어: 0.5893, PR score : 0.1530
임곗값: 0.1
오차행렬
[[2175   54]
 [  32   18]]
정확도: 0.9623, 정밀도 : 0.2500, 재현율:0.3600,F1 스코어:0.2951
ROC 스코어: 0.6679, PR score : 0.1040
임곗값: 0.2
오차행렬
[[2214   15]
 [  37   13]]
정확도: 0.9772, 정밀도 : 0.4643, 재현율:0.2600,F1 스코어:0.3333
ROC 스코어: 0.6266, PR score : 0.1369
임곗값: 0.3
오차행렬
[[2222    7]
 [  40   10]]
정확도: 0.9794, 정밀도 : 0.5882, 재현율:0.2000,F1 스코어:0.2985
ROC 스코어: 0.5984, PR score : 0.1352
임곗값: 0.4
오차행렬
[[2224    5]
 [  41    9]]
정확도: 0.9798, 정밀도 : 0.6429, 재현율:0.1800,F1 스코어:0.2812
ROC 스코어: 0.5889, PR score : 0.1337
임곗값: 0.5
오차행렬
[[2226    3]
 [  41    9]]
정확도: 0.9807, 정밀도 : 0.7500, 재현율:0.1800,F1 스코어:0.2903
ROC 스코어: 0.5893, PR score : 0.1530
임곗값: 0.6
오차행렬
[[2227    2]
 [  44    6]]
정확도: 0.9798, 정밀도 : 0.7500, 재현율:0.1200,F1 스코어:0.2069
ROC 스코어: 0.5596, PR score : 0.1093
임곗값: 0.7
오차행렬
[[2228    1]
 [  48    2]]
정확도: 0.9785, 정밀도 : 0.6667, 재현율:0.0400,F1 스코어:0.07

In [ ]:
# 1-5 xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 

xgb_wrapper = XGBClassifier()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    xgb_wrapper.fit(x_train, y_train, verbose = True)
    xgb_wrapper.fit(x_train, y_train)
    xgb_pred = xgb_wrapper.predict(x_test)
    xgb_pred_proba = xgb_wrapper.predict_proba(x_test)
    print(feature,"XGB") 
    get_clf_eval(y_test, xgb_pred)
    get_eval_by_threshold(y_test , xgb_pred_proba[:,1].reshape(-1,1), thresholds)

mda XGB
오차행렬
[[2221    8]
 [  41    9]]
정확도: 0.9785, 정밀도 : 0.5294, 재현율:0.1800,F1 스코어:0.2687
ROC 스코어: 0.5882, PR score : 0.1133
임곗값: 0.1
오차행렬
[[2207   22]
 [  36   14]]
정확도: 0.9746, 정밀도 : 0.3889, 재현율:0.2800,F1 스코어:0.3256
ROC 스코어: 0.6351, PR score : 0.1247
임곗값: 0.2
오차행렬
[[2213   16]
 [  37   13]]
정확도: 0.9767, 정밀도 : 0.4483, 재현율:0.2600,F1 스코어:0.3291
ROC 스코어: 0.6264, PR score : 0.1328
임곗값: 0.3
오차행렬
[[2217   12]
 [  38   12]]
정확도: 0.9781, 정밀도 : 0.5000, 재현율:0.2400,F1 스코어:0.3243
ROC 스코어: 0.6173, PR score : 0.1367
임곗값: 0.4
오차행렬
[[2221    8]
 [  39   11]]
정확도: 0.9794, 정밀도 : 0.5789, 재현율:0.2200,F1 스코어:0.3188
ROC 스코어: 0.6082, PR score : 0.1445
임곗값: 0.5
오차행렬
[[2221    8]
 [  41    9]]
정확도: 0.9785, 정밀도 : 0.5294, 재현율:0.1800,F1 스코어:0.2687
ROC 스코어: 0.5882, PR score : 0.1133
임곗값: 0.6
오차행렬
[[2221    8]
 [  41    9]]
정확도: 0.9785, 정밀도 : 0.5294, 재현율:0.1800,F1 스코어:0.2687
ROC 스코어: 0.5882, PR score : 0.1133
임곗값: 0.7
오차행렬
[[2221    8]
 [  41    9]]
정확도: 0.9785, 정밀도 : 0.5294, 재현율:0.1800,F1 스코어:0.2687
ROC 스코어: 0.5

In [ ]:
values[1]

'l2'

In [ ]:
#xgb 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

xgb_wrapper
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {
            'n_estimators':[40,43,46,50],
            'learning_rate':[0.03,0.05,0.04],
            'max_depth' : [0.8,1,2,3,4]
            }

    grid_clf = GridSearchCV(xgb_wrapper, param_grid=params, scoring='accuracy', cv=5)
    grid_clf.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

    a = grid_clf.best_params_
    keys=list(a.keys())
    values = list(a.values())
    xgb_clf_best = XGBClassifier( learning_rate=values[0],max_depth=values[1],n_estimators= values[2])
    xgb_clf_best.fit(x_train, y_train)
    xgb_best_pred = xgb_clf_best.predict(x_test)
    xgb_best_pred_proba = xgb_clf_best.predict_proba(x_test)   

    print(feature,"XGB 파라미터적용") 
    get_clf_eval(y_test, xgb_best_pred)
    get_eval_by_threshold(y_test ,xgb_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미어:{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 40}, 최적 평균 정확도:0.985
mda XGB 파라미터적용
오차행렬
[[2224    5]
 [  41    9]]
정확도: 0.9798, 정밀도 : 0.6429, 재현율:0.1800,F1 스코어:0.2812
ROC 스코어: 0.5889, PR score : 0.1337
임곗값: 0.1
오차행렬
[[2066  163]
 [  19   31]]
정확도: 0.9201, 정밀도 : 0.1598, 재현율:0.6200,F1 스코어:0.2541
ROC 스코어: 0.7734, PR score : 0.1074
임곗값: 0.2
오차행렬
[[2198   31]
 [  33   17]]
정확도: 0.9719, 정밀도 : 0.3542, 재현율:0.3400,F1 스코어:0.3469
ROC 스코어: 0.6630, PR score : 0.1349
임곗값: 0.3
오차행렬
[[2216   13]
 [  38   12]]
정확도: 0.9776, 정밀도 : 0.4800, 재현율:0.2400,F1 스코어:0.3200
ROC 스코어: 0.6171, PR score : 0.1319
임곗값: 0.4
오차행렬
[[2218   11]
 [  38   12]]
정확도: 0.9785, 정밀도 : 0.5217, 재현율:0.2400,F1 스코어:0.3288
ROC 스코어: 0.6175, PR score : 0.1419
임곗값: 0.5
오차행렬
[[2224    5]
 [  41    9]]
정확도: 0.9798, 정밀도 : 0.6429, 재현율:0.1800,F1 스코어:0.2812
ROC 스코어: 0.5889, PR score : 0.1337
임곗값: 0.6
오차행렬
[[2224    5]
 [  42    8]]
정확도: 0.9794, 정밀도 : 0.6154, 재현율:0.1600,F1 스코어:0.2540
ROC 스코어: 0.5789, PR score : 0.1169
임곗값: 0.7
